# Семантический Анализ Новосте и Их Влияние на Фондовою Биржу

In [28]:
from textblob import TextBlob
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd

### Таблица с данными

In [29]:
news_analysis = pd.DataFrame(columns=['news', 'date'])

### Парсинг сайта

In [30]:
URL = 'https://www.ft.com/companies/oil-gas?page=96'
first_page = 96
last_page = 138

In [31]:
our_soup = []
for page in range(first_page, last_page + 1):
    URL = f'https://www.ft.com/companies/oil-gas?page={str(page)}'
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'lxml')
    our_soup.append(soup)

In [32]:
for page in our_soup:
    cards = page.find_all("li", {'class':"o-teaser-collection__item o-grid-row"})
    for card in cards:
        try:
            date_of_article = card.find("time", {'class':"o-date"}).get_text()
            article = card.find("a", {'class':"js-teaser-heading-link"}).get_text()
            news_analysis.loc[len(news_analysis)] = [article, date_of_article]
        except AttributeError:
            pass

In [33]:
news_analysis

,news,date
0,Exxon and Chevron plan for Permian shale boom,"Tuesday, 5 March, 2019"
1,Pemex’s credit rating outlook chopped by Stand...,"Monday, 4 March, 2019"
2,Profits at Russia’s Lukoil double on higher oi...,"Monday, 4 March, 2019"
3,Price of plastic falls as demand in China slows,"Monday, 4 March, 2019"
4,Oil lobby group opposes carbon tax backed by b...,"Monday, 4 March, 2019"
...,...,...
1070,Shell warns of future LNG supply crunch,"Monday, 26 February, 2018"
1071,"With Brazilian biofuels on the rise, can we ke...","Monday, 26 February, 2018"
1072,Green energy tariffs could be exempt from Ofge...,"Monday, 26 February, 2018"
1073,Winners and losers in the age of energy abunda...,"Monday, 26 February, 2018"


### Семантический анализ

In [34]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report
from finbert.finbert import *
import finbert.utils as tools

from transformers import AutoModelForSequenceClassification


### Ссылка на уже обученную модель
https://huggingface.co/ProsusAI/finbert

In [35]:
model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert', cache_dir=None, num_labels=3)

In [36]:
text = 'Sultan Al Jaber: changing the mindset of a 50-year-old institution'.lower()

In [37]:
result = predict(text,model)

05/16/2022 00:04:05 - INFO - root -   Using device: cpu 
05/16/2022 00:04:05 - INFO - finbert.utils -   *** Example ***
05/16/2022 00:04:05 - INFO - finbert.utils -   guid: 0
05/16/2022 00:04:05 - INFO - finbert.utils -   tokens: [CLS] sultan al ja ##ber : changing the minds ##et of a 50 - year - old institution [SEP]
05/16/2022 00:04:05 - INFO - finbert.utils -   input_ids: 101 7544 2632 14855 5677 1024 5278 1996 9273 3388 1997 1037 2753 1011 2095 1011 2214 5145 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:04:05 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:04:05 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:04:05 - INFO - finbert.utils -   label: No

In [52]:
result

,sentence,logit,prediction,sentiment_score
0,trump’s plans for energy dominance meet resist...,"[0.31895205, 0.013194062, 0.6678539]",neutral,0.305758


 ### Финальное исследование

In [45]:
news_analysis = pd.DataFrame(columns=['news', 'date', 'prediction', 'sentiment_score'])

In [51]:
for page in our_soup:
    cards = page.find_all("li", {'class':"o-teaser-collection__item o-grid-row"})
    for card in cards:
        try:
            date_of_article = card.find("time", {'class':"o-date"}).get_text()
            article = card.find("a", {'class':"js-teaser-heading-link"}).get_text()
            result = predict(article.lower() ,model)
            prediction = result['prediction'].values[0]
            sentiment_score = result['sentiment_score'].values[0]
            news_analysis.loc[len(news_analysis)] = [article, date_of_article, prediction, sentiment_score]
        except AttributeError:
            pass

05/16/2022 00:17:54 - INFO - root -   Using device: cpu 
05/16/2022 00:17:54 - INFO - finbert.utils -   *** Example ***
05/16/2022 00:17:54 - INFO - finbert.utils -   guid: 0
05/16/2022 00:17:54 - INFO - finbert.utils -   tokens: [CLS] ex ##xon and che ##vron plan for per ##mian shale boom [SEP]
05/16/2022 00:17:54 - INFO - finbert.utils -   input_ids: 101 4654 22500 1998 18178 28588 2933 2005 2566 20924 18488 8797 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:17:54 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:17:54 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:17:54 - INFO - finbert.utils -   label: None (id = 9090)
05/16/2022 00:17:54 - 

05/16/2022 00:17:56 - INFO - finbert.utils -   label: None (id = 9090)
05/16/2022 00:17:56 - INFO - root -   tensor([[ 1.9662, -2.6113, -0.1686]])
05/16/2022 00:17:56 - INFO - root -   Using device: cpu 
05/16/2022 00:17:56 - INFO - finbert.utils -   *** Example ***
05/16/2022 00:17:56 - INFO - finbert.utils -   guid: 0
05/16/2022 00:17:56 - INFO - finbert.utils -   tokens: [CLS] hedge fund blue ##mount ##ain looks to replace pg & e board [SEP]
05/16/2022 00:17:56 - INFO - finbert.utils -   input_ids: 101 17834 4636 2630 20048 8113 3504 2000 5672 18720 1004 1041 2604 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:17:56 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:17:56 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

05/16/2022 00:17:57 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:17:57 - INFO - finbert.utils -   label: None (id = 9090)
05/16/2022 00:17:57 - INFO - root -   tensor([[ 1.4833, -2.6253,  0.5679]])
05/16/2022 00:17:57 - INFO - root -   Using device: cpu 
05/16/2022 00:17:57 - INFO - finbert.utils -   *** Example ***
05/16/2022 00:17:57 - INFO - finbert.utils -   guid: 0
05/16/2022 00:17:57 - INFO - finbert.utils -   tokens: [CLS] why venezuela needs an oil - for - food programme [SEP]
05/16/2022 00:17:57 - INFO - finbert.utils -   input_ids: 101 2339 8326 3791 2019 3514 1011 2005 1011 2833 4746 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:17:57 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

05/16/2022 00:17:59 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:17:59 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:17:59 - INFO - finbert.utils -   label: None (id = 9090)
05/16/2022 00:17:59 - INFO - root -   tensor([[ 0.2183,  0.6832, -1.6560]])
05/16/2022 00:17:59 - INFO - root -   Using device: cpu 
05/16/2022 00:17:59 - INFO - finbert.utils -   *** Example ***
05/16/2022 00:17:59 - INFO - finbert.utils -   guid: 0
05/16/2022 00:17:59 - INFO - finbert.utils -   tokens: [CLS] oil block ##chai ##n platform signs up most north sea traders [SEP]
05/16/2022 00:17:59 - INFO - finbert.utils -   input_ids: 101 3514 3796 24925 2078 4132 5751 2039 2087 2167 2712 13066 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

05/16/2022 00:18:00 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:18:00 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:18:00 - INFO - finbert.utils -   label: None (id = 9090)
05/16/2022 00:18:01 - INFO - root -   tensor([[-1.6799,  2.9257, -0.9437]])
05/16/2022 00:18:01 - INFO - root -   Using device: cpu 
05/16/2022 00:18:01 - INFO - finbert.utils -   *** Example ***
05/16/2022 00:18:01 - INFO - finbert.utils -   guid: 0
05/16/2022 00:18:01 - INFO - finbert.utils -   tokens: [CLS] wall street groups to invest $ 5 ##bn in abu dhabi pipeline unit [SEP]
05/16/2022 00:18:01 - INFO - finbert.utils -   input_ids: 101 2813 2395 2967 2000 15697 1002 1019 24700 1999 8273 23153 13117 3131 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

...

05/16/2022 00:21:31 - INFO - finbert.utils -   *** Example ***
05/16/2022 00:21:31 - INFO - finbert.utils -   guid: 0
05/16/2022 00:21:31 - INFO - finbert.utils -   tokens: [CLS] tighter power network regulation will pay off for uk consumers [SEP]
05/16/2022 00:21:31 - INFO - finbert.utils -   input_ids: 101 12347 2373 2897 7816 2097 3477 2125 2005 2866 10390 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:21:31 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:21:31 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:21:31 - INFO - finbert.utils -   label: None (id = 9090)
05/16/2022 00:21:31 - INFO - root -   tensor([[ 1.6736, -1.5241, -0.7132]])
0

05/16/2022 00:21:32 - INFO - finbert.utils -   label: None (id = 9090)
05/16/2022 00:21:32 - INFO - root -   tensor([[-1.3975,  2.3496, -0.3343]])
05/16/2022 00:21:32 - INFO - root -   Using device: cpu 
05/16/2022 00:21:32 - INFO - finbert.utils -   *** Example ***
05/16/2022 00:21:32 - INFO - finbert.utils -   guid: 0
05/16/2022 00:21:32 - INFO - finbert.utils -   tokens: [CLS] venezuela is the one to watch on oil [SEP]
05/16/2022 00:21:32 - INFO - finbert.utils -   input_ids: 101 8326 2003 1996 2028 2000 3422 2006 3514 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:21:32 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:21:32 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

05/16/2022 00:21:34 - INFO - finbert.utils -   label: None (id = 9090)
05/16/2022 00:21:34 - INFO - root -   tensor([[ 0.1970, -1.3617,  1.6287]])
05/16/2022 00:21:34 - INFO - root -   Using device: cpu 
05/16/2022 00:21:34 - INFO - finbert.utils -   *** Example ***
05/16/2022 00:21:34 - INFO - finbert.utils -   guid: 0
05/16/2022 00:21:34 - INFO - finbert.utils -   tokens: [CLS] china energy group bonds crash after detention report [SEP]
05/16/2022 00:21:34 - INFO - finbert.utils -   input_ids: 101 2859 2943 2177 9547 5823 2044 12345 3189 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:21:34 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:21:34 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

05/16/2022 00:21:35 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:21:35 - INFO - finbert.utils -   label: None (id = 9090)
05/16/2022 00:21:35 - INFO - root -   tensor([[-1.4720,  2.4693, -0.4536]])
05/16/2022 00:21:35 - INFO - root -   Using device: cpu 
05/16/2022 00:21:35 - INFO - finbert.utils -   *** Example ***
05/16/2022 00:21:35 - INFO - finbert.utils -   guid: 0
05/16/2022 00:21:35 - INFO - finbert.utils -   tokens: [CLS] swiss trader seeks go - ahead to buy venezuela - russia oil loan [SEP]
05/16/2022 00:21:35 - INFO - finbert.utils -   input_ids: 101 5364 17667 11014 2175 1011 3805 2000 4965 8326 1011 3607 3514 5414 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:21:35 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

05/16/2022 00:21:37 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:21:37 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2022 00:21:37 - INFO - finbert.utils -   label: None (id = 9090)
05/16/2022 00:21:37 - INFO - root -   tensor([[ 0.8493, -2.3359,  1.5884]])


In [70]:
news_analysis

,news,date,prediction,sentiment_score
0,Exxon and Chevron plan for Permian shale boom,"Tuesday, 5 March, 2019",neutral,0.059504
1,Pemex’s credit rating outlook chopped by Stand...,"Monday, 4 March, 2019",neutral,0.046552
2,Profits at Russia’s Lukoil double on higher oi...,"Monday, 4 March, 2019",negative,-0.50861
3,Price of plastic falls as demand in China slows,"Monday, 4 March, 2019",negative,-0.962134
4,Oil lobby group opposes carbon tax backed by b...,"Monday, 4 March, 2019",neutral,-0.073479
...,...,...,...,...
1070,Shell warns of future LNG supply crunch,"Monday, 26 February, 2018",negative,-0.914506
1071,"With Brazilian biofuels on the rise, can we ke...","Monday, 26 February, 2018",neutral,0.427458
1072,Green energy tariffs could be exempt from Ofge...,"Monday, 26 February, 2018",neutral,0.024513
1073,Winners and losers in the age of energy abunda...,"Monday, 26 February, 2018",neutral,0.015688


In [73]:
news_analysis.to_csv('news_analysis.csv')
news_analysis.to_excel('news_analysis.xlsx')